In [1]:
import tensorflow as tf

In [2]:
import numpy as np

In [3]:
import matplotlib.pyplot as plt

In [4]:
%matplotlib inline

In [5]:
from sklearn.utils import shuffle

In [9]:
#teraz sproboje z tego zrobic jakis dataset

In [10]:
#f,l=kolko_w_kolku()

In [11]:
#dataset = tf.data.Dataset.from_tensor_slices((f,l))

In [12]:
#BATCH_SIZE=100
#zbachowany=dataset.shuffle(1000).repeat().batch(BATCH_SIZE)

In [13]:
#to jest dataset zaraz zastanowie sie jak go zapisac do pliku i z tego pliku odczytac a teraz jak go czytac?

In [14]:
#iterator = zbachowany.make_one_shot_iterator()

In [15]:
#f,l=iterator.get_next()

In [16]:
#with tf.Session() as sess:
#    for i in range(100):
#        sess.run(l)
    

In [17]:
#wrap_float64([5.55,6.85]) #swietnie

In [12]:


"""

Podajemy slownik na takich samych zasadach co podawalismy do tamtego Io_tf_binary
Tym razem jak podajemy sciezke do pliku to ten plik ma juz byc pelen danych. 

Na razie w pierwszym rzucie nie bedzie jeszcze danych typu kategorycznego.

To ma byc kompatybilne z klasa Io_tf_binary_general

__init__(nazwa_folderu,hidden_units,model_dir)
        to jest nazwa_folderu odnosi sie do folderu w ktorym pisala klasa
        Io_tf_binary_general czy cos
        hidden_units to jest lista po ile ma byc ukrytych unitsow. czyli nie podajemy rozmiaru
        danych wejsciowych ani wyjsciowych. idziemy od pierwszej (najblizszej inputu) do ostatniej
        model_dir to tam bedzie pisac swoje rzeczy nasz model




"""

import Io_tf_binary_general as io


class Dnn_uniwersalny:
    def __init__(self,nazwa_folderu,hidden_units,model_dir):
        self.model_dit=model_dir
        self.hidden_units=hidden_units
        self.wczytywacz=io.Io_tf_binary_general(nazwa_folderu,'r')
        self.typy=self.wczytywacz.types()
        my_feature_columns = []
        for k in self.typy.keys():
            if self.typy[k][1]=='f':
                t=tf.float32
            else:
                t=tf.int32
            my_feature_columns.append(tf.feature_column.numeric_column(key=k,shape=\
                        (self.typy[k][0],),dtype=t ))
        
        self.feature_columns=my_feature_columns
        
        def input_fn( batch_size=100,buffer_size=1000):
            """input function for training
            nazwy to lista nazw feature w kolejnosci wystepowania"""
            dataset=self.wczytywacz.read()
            return dataset.shuffle(buffer_size).repeat().batch(batch_size)
        self.input_fn=input_fn
        
        self.classifier = tf.estimator.DNNClassifier(
        feature_columns=self.feature_columns,
        hidden_units=self.hidden_units,
        model_dir=model_dir,
        n_classes=2)
    def train(self,batch_size=128,buffer_size=1000,steps=3000):
        self.classifier.train(
        input_fn=lambda:self.input_fn( batch_size,buffer_size),
        steps=steps)
    def evaluate(self,batch_size=128,buffer_size=1000,steps=1000):
        self.last_eval_result = self.classifier.evaluate(
        input_fn=lambda:self.input_fn( batch_size,buffer_size),
        steps=steps)
        return self.last_eval_result
    
    
    
    
        
        
        
    
        

In [13]:
#Ta linijka zaklada, ze zrobilismy plik dane_treningowe2 wedlug Io_tf_binary

In [14]:
model=Dnn_uniwersalny('weak_train',[10],"weak_estimator")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'weak_estimator', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa64d925208>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [15]:
model.train()

{'pierwsza': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'inna_wlasnosc': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'label': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None)}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into weak_estimator/model.ckpt.
INFO:tensorflow:loss = 91.72041, step = 1
INFO:tensorflow:global_step/sec: 159.83
INFO:tensorflow:loss = 84.346756, step = 101 (0.627 sec)
INFO:tensorflow:global_step/sec: 183.737
INFO:tensorflow:loss = 83.75353, step = 201 (0.544 sec)
INFO:tensorflow:global_step/sec: 194.471
INFO:tensorflow:loss = 84.53551, step = 301 (0.516 sec)
INFO:tensorflow:global_step/sec: 191.504
INFO:tensorflow:loss = 88.83089, step = 401 (0.520 sec)
INFO:tensorflow:global_ste

In [16]:
model.evaluate()

{'pierwsza': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'inna_wlasnosc': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'label': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None)}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-13-13:37:59
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from weak_estimator/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [100/1000]
INFO:tensorflow:Evaluation [200/1000]
INFO:tensorflow:Evaluation [300/1000]
INFO:tensorflow:Evaluation [400/1000]
INFO:tensorflow:Evaluation [500/1000]
INFO:tensorflow:Evaluation [600/1000]
INFO:tensorflow:Evaluation [700/1000]
INFO:tensorflow:Evaluation [800/1000]
INFO:tensorflow:Evaluation [900/1000]
INFO:tensorflow:Evaluation [1000/1000]
INFO:tensorflow:Finished evaluation at 2018-05-13-13:38:07
I

{'accuracy': 0.6055781,
 'accuracy_baseline': 0.5031328,
 'auc': 0.6215453,
 'auc_precision_recall': 0.59210587,
 'average_loss': 0.6676708,
 'label/mean': 0.5031328,
 'loss': 85.46186,
 'precision': 0.5866298,
 'prediction/mean': 0.5049913,
 'recall': 0.7315725,
 'global_step': 3000}